In [ ]:
import numpy as np
import cv2
import torch
import torch.nn as nn
import torch.optim as optim
import torch.nn.functional as F
from torchvision import transforms

In [ ]:
num_classes = 10
model_name = 'pytorch_cifar10_model.pth'
device = 'cuda:0'

In [ ]:
# network definition
class Classifier(nn.Module):
    def __init__(self):
        super().__init__()
        self.encoder = nn.Sequential(
            nn.Conv2d(3,32,(3,3),padding=1),
            nn.ReLU(),
            nn.Conv2d(32,32,(3,3)),
            nn.ReLU(),
            nn.MaxPool2d(2),
            nn.Dropout(0.25),
            #----------------
            nn.Conv2d(32,64,(3,3),padding=1),
            nn.ReLU(),
            nn.Conv2d(64,64,(3,3)),
            nn.ReLU(),
            nn.MaxPool2d(2),
            nn.Dropout(0.25),
            #----------------
            nn.Conv2d(64,64,(3,3),padding=1),
            nn.ReLU(),
            nn.Conv2d(64,64,(3,3)),
            nn.ReLU(),
            nn.MaxPool2d(2),
            nn.Dropout(0.25),
            #----------------
            nn.Flatten()
        )
        self.perceptron = nn.Sequential(
            nn.Linear(256,512),
            nn.ReLU(),
            nn.Dropout(0.5),
            nn.Linear(512,num_classes),
            nn.Softmax(1)
        )
    def forward(self, x):
        features = self.encoder(x)
        return self.perceptron(features)

In [ ]:
model = Classifier()

In [ ]:
from google.colab import files
print('upload',model_name)
_ = files.upload()

In [ ]:
model.load_state_dict(torch.load(model_name))

In [ ]:
model.to(device)
model.eval()

In [ ]:
!wget http://www.agentspace.org/download/cifarsamples.zip
!unzip -q cifarsamples.zip
!rm cifarsamples.zip
!ls -l *.png

In [ ]:
image1 = cv2.imread('car.png',cv2.IMREAD_COLOR) # from dataset
image2 = cv2.imread('cat.png',cv2.IMREAD_COLOR) # from dataset
image3 = cv2.imread('cat2.png',cv2.IMREAD_COLOR) # from internet & resized

In [ ]:
image1 = cv2.cvtColor(image1,cv2.COLOR_BGR2RGB)
image2 = cv2.cvtColor(image2,cv2.COLOR_BGR2RGB)
image3 = cv2.cvtColor(image3,cv2.COLOR_BGR2RGB)

In [ ]:
image3.min(),image3.max()

In [ ]:
transform = transforms.Compose([transforms.ToTensor(),transforms.Normalize((0.4914, 0.4822, 0.4465),(0.2023, 0.1994, 0.2010))])
input_images = torch.stack((transform(image1),transform(image2),transform(image3))).to(device)

In [ ]:
input_images[0][0].min(),input_images[0][0].max()

In [ ]:
categories = model(input_images)

In [ ]:
classes = ('plane', 'car', 'bird', 'cat', 'deer', 'dog', 'frog', 'horse', 'ship', 'truck')
print([classes[i] for i in torch.argmax(categories,dim=1)])
print([f"{probability.item():.3f}" for probability in torch.max(categories,dim=1)[0]])